# Run a training script as a command job

You can use the Python SDK for Azure Machine Learning to submit scripts as command jobs. By using jobs, you can easily keep track of the input parameters and outputs when training a machine learning model.

## Before you start

You'll need the latest version of the  **azureml-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [ ]:
pip show azure-ai-ml

## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

> **Note**: When working from a notebook on an Azure Machine Learning managed compute instance, you don't have to provide the values for the subscription ID, resource group, and workspace name. The values are retrieved from the workspace you currently work in.

In [ ]:
# import necessary packages
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azureml.core import Workspace

# get details of the current Azure ML workspace
ws = Workspace.from_config()

# default authentication flow for Azure applications
default_azure_credential = DefaultAzureCredential()
subscription_id = ws.subscription_id
resource_group = ws.resource_group
workspace = ws.name

# client class to interact with Azure ML services and resources, e.g. workspaces, jobs, models and so on.
ml_client = MLClient(
    default_azure_credential,
    subscription_id,
    resource_group,
    workspace)

print('Ready to work with {}'.format(ws.name)) 

## Initiate a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train-model-parameters.py** script that can be found in the **src** folder. It uses the **diabetes.csv** file as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command used to test the script in the terminal is the same as the command in the configuration of the job below. 

In [ ]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-parameters.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-script",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)